In [51]:

#importing Libraries
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

ModuleNotFoundError: No module named 'shapely'

### Requesting Url and getting data from url

In [52]:
!pip install shapely

     |████████████████████████████████| 1.8 MB 3.2 MB/s eta 0:00:01


In [20]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

### First 5 records

In [21]:
data.head()


,Borough,Neighbourhood,Postal Code
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A


### Choosing only data where field Borough doesn't have not assigned value


In [24]:
raw_data = data[data['Borough'] != 'Not assigned']

In [27]:
raw_data = raw_data.groupby(['Borough', 'Postal Code'], as_index=False).agg(','.join)

In [28]:
raw_data.head()

,Borough,Postal Code,Neighbourhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


### Replacing values in Neighbourhood field with Borough where Neighbourhood is not assigned

In [30]:
raw_data['Neighbourhood'] = np.where(raw_data['Neighbourhood'] == 'Not assigned', raw_data['Borough'], raw_data['Neighbourhood'])

In [32]:
raw_data.shape

(103, 3)

In [33]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [34]:
geospatial_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
merge_data = pd.merge(geospatial_data,raw_data,on='Postal Code')

In [41]:
merge_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


# Q3

In [44]:
merge_data['Coordinates'] = list(zip(merge_data['Latitude'], merge_data['Longitude']))

In [46]:
merge_data.head()

,Postal Code,Latitude,Longitude,Borough,Neighbourhood,Coordinates
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge","(43.806686299999996, -79.19435340000001)"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek","(43.7845351, -79.16049709999999)"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill","(43.7635726, -79.1887115)"
3,M1G,43.770992,-79.216917,Scarborough,Woburn,"(43.7709921, -79.21691740000001)"
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae,"(43.773136, -79.23947609999999)"


In [50]:
merge_data['Coordinates'] = merge_data['Coordinates'].apply(Point)

AttributeError: 'DataFrame' object has no attribute 'DataFrame'